# Process results

This notebook analyzes the matching decisions by the simulation.py and summarizes a few metrics

In [1]:
import pandas as pd
import numpy as np

In [13]:
recipients = pd.read_csv('data/test/test_recipients.csv',index_col=0,parse_dates=['date'])
recipients = recipients[recipients['qty']>0]
recipients

,rec_id,date,ppe,qty
0,R1,2020-04-04 13:08:00+00:00,masks,100.0
1,R1,2020-04-09 13:08:00+00:00,gloves,300.0
2,R2,2020-04-10 13:08:00+00:00,gloves,200.0
3,R2,2020-04-12 13:08:00+00:00,gowns,50.0
4,R3,2020-04-05 13:08:00+00:00,gowns,60.0


In [14]:
donors = pd.read_csv('data/test/test_donors.csv',index_col=0,parse_dates=['date'])
donors

,don_id,date,ppe,qty,don_req_id
0,D1,2020-04-05 13:08:00+00:00,gloves,400,0
1,D1,2020-04-11 13:08:00+00:00,gowns,20,1
2,D2,2020-04-12 13:08:00+00:00,gowns,30,2
3,D2,2020-04-12 13:08:00+00:00,masks,130,3


In [15]:
import pickle
decisions = pd.read_csv('data/test/test_decisions.csv',index_col=0,parse_dates=['date'])
decisions

,date,don_id,rec_id,ppe,qty,distance
0,2020-04-15 13:08:00+00:00,D1,R1,gloves,300,100
1,2020-04-15 13:08:00+00:00,D1,R2,gloves,100,100
2,2020-04-15 13:08:00+00:00,D1,R2,gowns,20,100
3,2020-04-20 13:08:00+00:00,D2,R1,masks,130,50
4,2020-04-20 13:08:00+00:00,D2,R2,gowns,30,50


In [16]:
all_ppes = set(donors.ppe.unique())
all_ppes = all_ppes.union(set(recipients.ppe.unique()))

In [17]:
#ppes_to_consider = set(donors.ppe.unique())
#ppes_to_consider = ppes_to_consider.intersection(set(recipients.ppe.unique()))

# Set up result table

In [41]:
result = pd.DataFrame(columns=['metric_name','description','value'])
result

,metric_name,description,value


# Fill rate for rec_id,ppe

In [35]:
fr = recipients.groupby(['rec_id','ppe'])['qty'].agg(['sum','size'])
fr = fr.reset_index()
fr.columns=['rec_id','ppe','requested', 'fill_rate']
fr['fill_rate'] = -1
fr

,rec_id,ppe,requested,fill_rate
0,R1,gloves,300.0,-1
1,R1,masks,100.0,-1
2,R2,gloves,200.0,-1
3,R2,gowns,50.0,-1
4,R3,gowns,60.0,-1


In [42]:
for i,row in fr.iterrows():
    rec = row.rec_id
    ppe = row.ppe
    # find proportion of ppe received by rec out of the ones requested
    received = decisions.loc[(decisions['rec_id'] == rec) & (decisions['ppe'] == ppe),'qty'].sum()
    #print(f'{rec} received {received} units of {ppe} out of {row.requested} units requested')
    val = 0 if received == 0 else min(1,received/row.requested)
    fr.loc[i,'fill_rate'] = val
    result.loc[len(result)] = [f'fill rate ({rec},{ppe})', f'fill rate of recipient {rec} limited to {ppe}',val]
fr

,rec_id,ppe,requested,fill_rate
0,R1,gloves,300.0,1.0
1,R1,masks,100.0,1.0
2,R2,gloves,200.0,0.5
3,R2,gowns,50.0,1.0
4,R3,gowns,60.0,0.0


In [43]:
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.5
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0


# Fill rate for each ppe

For each ppe, the average fill rate from table fr

In [32]:
fr_p = fr.groupby('ppe')['fill_rate'].mean()
fr_p

ppe
gloves    0.75
gowns     0.50
masks     1.00
Name: fill_rate, dtype: float64

In [46]:
for ppe,val in fr_p.items():
    result.loc[len(result)] = [f'fill rate ({ppe})', f'average fill rate among recipients who requested {ppe}',val]

In [47]:
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.5
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0
5,fill rate (gloves),average fill rate among recipients who request...,0.75
6,fill rate (gowns),average fill rate among recipients who request...,0.5
7,fill rate (masks),average fill rate among recipients who request...,1


Excluding the zero fill_rates

In [33]:
fr_p_zero = fr[fr.fill_rate > 0].groupby('ppe')['fill_rate'].mean()
fr_p_zero

ppe
gloves    0.75
gowns     1.00
masks     1.00
Name: fill_rate, dtype: float64

In [48]:
for ppe,val in fr_p_zero.items():
    result.loc[len(result)] = [f'fill rate exc zeros ({ppe})', f'average fill rate among recipients who requested {ppe} and received at least one unit',val]
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.5
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0
5,fill rate (gloves),average fill rate among recipients who request...,0.75
6,fill rate (gowns),average fill rate among recipients who request...,0.5
7,fill rate (masks),average fill rate among recipients who request...,1
8,fill rate exc zeros (gloves),average fill rate among recipients who request...,0.75
9,fill rate exc zeros (gowns),average fill rate among recipients who request...,1


# overall fill rate

In [52]:
result.loc[len(result)] = [f'fill rate', f'overall fill rate, i.e., the average of the fill rates (ppe)',fr_p.mean()]
result.loc[len(result)] = [f'fill rate exc zeros', f'overall fill rate among recipients who received something, i.e., the average of the fill rates (ppe) among recipients who received at least one unit',fr_p_zero.mean()]



In [53]:
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.5
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0
5,fill rate (gloves),average fill rate among recipients who request...,0.75
6,fill rate (gowns),average fill rate among recipients who request...,0.5
7,fill rate (masks),average fill rate among recipients who request...,1
8,fill rate exc zeros (gloves),average fill rate among recipients who request...,0.75
9,fill rate exc zeros (gowns),average fill rate among recipients who request...,1


# Continue here with avg miles, holding time, etc

# For each ppe, compute stats

Fill rate

In [11]:
merg = recipients.merge(decisions,on=['rec_id','ppe'],how='left',suffixes=('_rec','_decision'))
merg

,rec_id,date_rec,ppe,qty_rec,date_decision,don_id,qty_decision,distance
0,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,NaT,NaN,NaN,NaN
1,rec0,2020-04-02 16:27:00+00:00,respirators,10000.0,NaT,NaN,NaN,NaN
2,rec0,2020-04-02 16:27:00+00:00,gowns,1000.0,NaT,NaN,NaN,NaN
3,rec0,2020-04-02 16:27:00+00:00,faceShields,5000.0,NaT,NaN,NaN,NaN
4,rec1,2020-04-02 16:35:00+00:00,disinfectingWipes,4.0,2020-04-23 16:26:00+00:00,don114,4.0,1.910903
...,...,...,...,...,...,...,...,...
37399,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0,NaT,NaN,NaN,NaN
37400,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0,NaT,NaN,NaN,NaN
37401,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0,NaT,NaN,NaN,NaN
37402,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0,NaT,NaN,NaN,NaN


In [12]:
merg['delay'] = (merg['date_decision'] - merg['date_rec']).dt.days
#merg = merg[merg['delay'] < threshold_days]

In [13]:
merg

,rec_id,date_rec,ppe,qty_rec,date_decision,don_id,qty_decision,distance,delay
0,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,NaT,NaN,NaN,NaN,NaN
1,rec0,2020-04-02 16:27:00+00:00,respirators,10000.0,NaT,NaN,NaN,NaN,NaN
2,rec0,2020-04-02 16:27:00+00:00,gowns,1000.0,NaT,NaN,NaN,NaN,NaN
3,rec0,2020-04-02 16:27:00+00:00,faceShields,5000.0,NaT,NaN,NaN,NaN,NaN
4,rec1,2020-04-02 16:35:00+00:00,disinfectingWipes,4.0,2020-04-23 16:26:00+00:00,don114,4.0,1.910903,20.0
...,...,...,...,...,...,...,...,...,...
37399,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0,NaT,NaN,NaN,NaN,NaN
37400,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0,NaT,NaN,NaN,NaN,NaN
37401,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0,NaT,NaN,NaN,NaN,NaN
37402,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0,NaT,NaN,NaN,NaN,NaN


In [14]:
res = merg.groupby(['rec_id','ppe']).agg({'qty_rec':min,'qty_decision':sum}).reset_index()
res['total_fill_rate'] = res.qty_decision / res.qty_rec
res

,rec_id,ppe,qty_rec,qty_decision,total_fill_rate
0,rec0,faceShields,5000.0,0.0,0.0
1,rec0,gowns,1000.0,0.0,0.0
2,rec0,nitrileGloves,10000.0,0.0,0.0
3,rec0,respirators,10000.0,0.0,0.0
4,rec1,disinfectingWipes,4.0,4.0,1.0
...,...,...,...,...,...
37299,rec998,respirators,50.0,0.0,0.0
37300,rec998,thermometers,2.0,0.0,0.0
37301,rec999,faceShields,20.0,0.0,0.0
37302,rec999,rec_req_id,999.0,0.0,0.0


In [15]:
res

,rec_id,ppe,qty_rec,qty_decision,total_fill_rate
0,rec0,faceShields,5000.0,0.0,0.0
1,rec0,gowns,1000.0,0.0,0.0
2,rec0,nitrileGloves,10000.0,0.0,0.0
3,rec0,respirators,10000.0,0.0,0.0
4,rec1,disinfectingWipes,4.0,4.0,1.0
...,...,...,...,...,...
37299,rec998,respirators,50.0,0.0,0.0
37300,rec998,thermometers,2.0,0.0,0.0
37301,rec999,faceShields,20.0,0.0,0.0
37302,rec999,rec_req_id,999.0,0.0,0.0


In [16]:
res.groupby('ppe')['total_fill_rate'].mean()

ppe
babyMonitors         0.005286
bodyBags             0.005435
coveralls            0.023564
disinfectingWipes    0.002841
disposableBooties    0.006105
faceShields          0.046250
gowns                0.000925
handSanitizer        0.003906
handmadeMasks        0.050814
nitrileGloves        0.006539
paprShield           0.000000
rec_req_id           0.000000
respirators          0.012295
safetyGlasses        0.008809
safetyGoggles        0.011034
surgicalCaps         0.001350
surgicalMasks        0.020832
thermometers         0.000722
Name: total_fill_rate, dtype: float64